In [2]:
pwd

'C:\\Users\\lenovo'

In [ ]:
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import cv2
import svgwrite
from colormap import rgb2hex

In [ ]:
def load_image(image_path):
    """Load image as RGB"""
    return np.array(Image.open(image_path).convert('RGB'))



In [ ]:
def load_images(folder_path):
    """Load images from folder as RGB"""
    images = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            images[filename] = np.array(Image.open(image_path).convert('RGB'))
    return images

In [ ]:
def calculate_optimal_clusters(image, max_clusters=10):
    """Calculate optimal number of clusters using Silhouette score"""
    w, h, d = image.shape
    image_array = np.reshape(image, (w * h, d))
    silhouette_scores = []
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters).fit(image_array)
        labels = kmeans.labels_
        score = silhouette_score(image_array, labels)
        silhouette_scores.append(score)
    optimal_clusters = np.argmax(silhouette_scores) + 2
    return optimal_clusters

In [ ]:
def apply_kmeans(image, n_clusters):
    """Apply KMeans clustering"""
    w, h, d = image.shape
    image_array = np.reshape(image, (w * h, d))
    kmeans = KMeans(n_clusters=n_clusters).fit(image_array)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    return labels, centers


In [ ]:
def get_dominant_color(contour, image):
    """Get dominant color of a contour"""
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    ROI = cv2.bitwise_and(image, image, mask=mask)
    pixels = ROI.reshape((-1, 3))
    pixels = pixels[pixels.sum(axis=1) > 0]  # Remove black pixels
    if len(pixels) > 0:
        dominant_color = np.mean(pixels, axis=0).astype(int)
    else:
        dominant_color = (0, 0, 0)  # Default to black if no pixels
    return dominant_color



In [ ]:
def get_contours(image, labels, centers):
    """Get contours for each cluster"""
    contours = {}
    w, h, d = image.shape
    unique_labels = np.unique(labels)
    for label in unique_labels:
        mask = labels == label
        contour = np.zeros(image.shape, dtype=np.uint8)
        contour[mask.reshape(image.shape[:2])] = (int(centers[label][0]), int(centers[label][1]), int(centers[label][2]))
        gray = cv2.cvtColor(contour, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
        contour, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        dominant_color = get_dominant_color(contour[0], image)
        hex_color = rgb2hex(dominant_color[0], dominant_color[1], dominant_color[2])
        contours[label] = {
            'contour': contour,
            'dominant_color': dominant_color,
            'hex_color': hex_color,
            'cluster_id': label  # Retain original cluster ID
        }
    return contours

In [ ]:
def draw_contours(image, contours, output_filename):
    """Draw contours on white background and save as SVG"""
    w, h, _ = image.shape
    svg_image = svgwrite.Drawing(output_filename, (w, h))
    svg_image.add(svgwrite.rect(insert=(0, 0), size=(w, h), fill='white'))
    for cluster_id, contour_info in contours.items():
        contour = contour_info['contour']
        hex_color = contour_info['hex_color']
        for c in contour:
            svg_image.add(svgwrite.polyline(points=c.flatten().tolist(), 
                                            stroke='black', 
                                            stroke_width=2, 
                                            fill='none'))
        x, y, w, h = cv2.boundingRect(contour[0])
        svg_image.add(svgwrite.text(f"Cluster {cluster_id + 1}: {hex_color}", 
                                    insert=(x, y - 10), 
                                    fill='black', 
                                    font_size=20))
    svg_image.save()




In [ ]:
def draw_color_palette(contours, output_filename):
    """Draw color palette and save as SVG"""
    w, h = 500, 100
    svg_palette = svgwrite.Drawing(output_filename, (w, h))
    svg_palette.add(svgwrite.rect(insert=(0, 0), size=(w, h), fill='white'))
    x = 50
    for cluster_id, contour_info in contours.items():
        hex_color = contour_info['hex_color']
        svg_palette.add(svgwrite.rect(insert=(x, 50), size=(50, 50), fill=hex_color))
        svg_palette.add(svgwrite.text(f"Cluster {cluster_id + 1}: {hex_color}", 
                                      insert=(x, 120), 
                                      fill='black', 
                                      font_size=20))
        x += 70
    svg_palette.save()



In [ ]:
def refine_contours(contours, threshold=5):
    """Refine contours by removing duplicate points and merging close points"""
    refined_contours = []
    for contour in contours:
        contour = np.array(contour)
        # Remove duplicate points
        contour = np.unique(contour, axis=0)

        merged_points = [contour[0][0]]  # Start with the first point
        for i in range(1, len(contour)):
            found_merge = False
            for j in range(i+1, len(contour)):
                dist = np.linalg.norm(contour[i][0] - contour[j][0])
                if dist < threshold:
                    # Check if points are on the same line
                    vec1 = contour[i-1][0] - contour[i][0]
                    vec2 = contour[j-1][0] - contour[j][0]
                    if np.abs(np.cross(vec1, vec2)).sum() < 0.1:
                        # Merge points
                        midpoint = (contour[i][0] + contour[j][0]) // 2
                        merged_points.append(midpoint)
                        found_merge = True
                        break
            if not found_merge:
                merged_points.append(contour[i][0])
        # Close the contour
        merged_points.append(merged_points[0])
        refined_contours.append(np.array([merged_points]))

    return refined_contours


In [ ]:
def gaussian_blur(image, kernel_size=(3, 3), sigma=0):
    """Apply Gaussian blur to an image"""
    blurred_image = cv2.GaussianBlur(image, kernel_size, sigma)
    return blurred_image


In [ ]:
def identify_parallel_lines(close_points, threshold_slope=0.1, threshold_dist=1):
    """Identify parallel lines corresponding to neighboring color segments"""
    parallel_lines = {}

    # Check if close_points is empty
    if not close_points:
        return parallel_lines

    # Iterate through close_points
    for pair, points in close_points.items():
        # Handle single-point case
        if len(points) < 2:
            continue

        # Calculate slope and distance for each pair of points
        for i in range(len(points)):
            for j in range(i + 1, len(points)):
                point1 = points[i]
                point2 = points[j]

                try:
                    # Calculate slope manually
                    dx = point2[0] - point1[0]
                    dy = point2[1] - point1[1]

                    # Check for zero distance
                    if dx == 0 and dy == 0:
                        continue

                    # Check for vertical line
                    if dx == 0:
                        slope = float('inf')
                    else:
                        slope = dy / dx

                    # Calculate distance
                    dist = np.linalg.norm(np.array([dx, dy]))

                    # Check conditions
                    condition1 = np.isfinite(slope) and abs(slope) <= threshold_slope
                    condition2 = dist <= threshold_dist

                    if condition1 and condition2:
                        # Add to parallel_lines dictionary
                        if pair not in parallel_lines:
                            parallel_lines[pair] = []
                        parallel_lines[pair].append((point1, point2))
                except ZeroDivisionError:
                    # Handle division by zero
                    continue
                except Exception as e:
                    # Handle other exceptions
                    print(f"Error: {e}")
                    continue

    return parallel_lines


In [ ]:
def canny_edge_detection(image, threshold1, threshold2):
    """Apply Canny Edge Detection"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1, threshold2)
    return edges


def bgr_canny(image, threshold1, threshold2):
    """Apply Canny Edge Detection on BGR channels and combine results"""
    # Split BGR channels
    b, g, r = cv2.split(image)

    # Apply Canny on each channel
    edges_b = canny_edge_detection(cv2.merge([b, b, b]), threshold1, threshold2)
    edges_g = canny_edge_detection(cv2.merge([g, g, g]), threshold1, threshold2)
    edges_r = canny_edge_detection(cv2.merge([r, r, r]), threshold1, threshold2)

    # Combine edges using bitwise OR
    combined_edges = cv2.bitwise_or(edges_b, edges_g)
    combined_edges = cv2.bitwise_or(combined_edges, edges_r)

    return combined_edges


In [ ]:
def multi_level_canny(image, thresholds=None):
    """Apply multi-level Canny detection"""
    combined_edges = None

    # Define multi-level thresholds
    if thresholds is None:
        thresholds = [
            (5, 60),
            (60, 110),
            (110, 160),
            (160, 254)
        ]

    for (threshold1, threshold2) in thresholds:
        edges = bgr_canny(image, threshold1, threshold2)
        if combined_edges is None:
            combined_edges = edges
        else:
            combined_edges = cv2.bitwise_or(combined_edges, edges)

    return combined_edges


In [ ]:
def multilevel_canny_hsv(image, thresholds=None):
    """Apply multilevel Canny edge detection on HSV images"""
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    if thresholds is None:
        thresholds = [
            (50, 80),  # Low-intensity edges
            (80, 150),  # Medium-intensity edges
            (150, 210)  # High-intensity edges
        ]

    # Define function to apply multilevel Canny
    def multilevel_single_canny(channel):
        edges = []
        for threshold1, threshold2 in thresholds:
            edge = cv2.Canny(channel, threshold1, threshold2)
            edges.append(edge)
        combined_edges = np.bitwise_or.reduce(edges)
        return combined_edges

    # Apply multilevel Canny on V and S channels
    v_channel = hsv[:, :, 2]
    s_channel = hsv[:, :, 1]

    edges_v = multilevel_single_canny(v_channel)
    edges_s = multilevel_single_canny(s_channel)

    return edges_v, edges_s


In [ ]:
def main(folder_path):
    images = load_images(folder_path)
    for filename, image in images.items():
        optimal_k = determine_optimal_k(image)
        blurred_image = gaussian_blur(image)
        edges_v, edges_s = multilevel_canny_hsv(blurred_image)
        combined_edges = cv2.bitwise_or(edges_v, edges_s)
        contours, _ = cv2.findContours(combined_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        refined_contours = refine_contours(contours)
        labels, centers = apply_kmeans(image, optimal_k)
        refined_contours_dict = {}
        for contour in refined_contours:
            x, y, w, h = cv2.boundingRect(contour)
            ROI = cv2.bitwise_and(image, image, mask=np.zeros(image.shape, dtype=np.uint8), roi=(x, y, w, h))
            ROI_labels, ROI_centers = apply_kmeans(ROI, optimal_k)
            ROI_contours = get_contours(ROI, ROI_labels, ROI_centers)
            refined_contours_dict.update(ROI_contours)
        output_filename = f"{filename.split('.')[0]}_contours_{optimal_k}.svg"
        draw_contours(image, refined_contours_dict, output_filename)
        output_filename = f"{filename.split('.')[0]}_palette_{optimal_k}.svg"
        draw_color_palette(refined_contours_dict, output_filename)
        print(f"Processed {filename} with {optimal_k} clusters")


In [ ]:
import os

if __name__ == "__main__":
    try:
        pwd = os.getcwd()  # Get current working directory
        image_folder_path = os.path.join(pwd, 'images')  # Append 'images' folder to get complete path
    except Exception as e:
        print(f"Error occurred: {e}")
        image_folder_path = './images'  # Default to relative path

    main(image_folder_path)